# Bigram Character-Level Language Model

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

#parameters
block_size = 8 #size of each context block
batch_size = 4 # number of blocks running in parallel
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250 # show loss data every eval_iters

cuda


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
chars = sorted(set(text))
vocab_size = len(chars)
print(vocab_size)
print(chars)

81
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [5]:
# character level tokenizer
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s] # encoder given string returns list of corresponding char integers
decode = lambda l: ''.join([int_to_string[i] for i in l]) # decoder given list of integers returns corresponding string

# examples
print(decode(encode('hello')))

hello


In [6]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([80,  0, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0, 26, 49,  0,  0,
        36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0, 25, 45, 44,
        32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1,
        39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25, 38, 28,  1, 39, 30,
         1, 39, 50,  9,  1, 39, 50, 37, 25,  1])


In [8]:
# splitting to training and validation data
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    # tensor of initial position of x with size batch_size
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y  = x.to(device), y.to(device) # pushing to gpu
    return x,y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets')
print(y)

inputs:
tensor([[58, 66,  1, 54, 73,  1, 68, 67],
        [ 1, 59, 68, 71,  1, 33,  1, 66],
        [72, 11,  1,  3, 49, 68, 74,  1],
        [62, 67,  1, 76, 61, 54, 73,  1]], device='cuda:0')
targets
tensor([[66,  1, 54, 73,  1, 68, 67, 56],
        [59, 68, 71,  1, 33,  1, 66, 58],
        [11,  1,  3, 49, 68, 74,  1, 66],
        [67,  1, 76, 61, 54, 73,  1, 61]], device='cuda:0')


In [9]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t] # next encoded character given context
    print( f'when input is {context} target is {target}')

when input is tensor([80]) target is 0
when input is tensor([80,  0]) target is 28
when input is tensor([80,  0, 28]) target is 39
when input is tensor([80,  0, 28, 39]) target is 42
when input is tensor([80,  0, 28, 39, 42]) target is 39
when input is tensor([80,  0, 28, 39, 42, 39]) target is 44
when input is tensor([80,  0, 28, 39, 42, 39, 44]) target is 32
when input is tensor([80,  0, 28, 39, 42, 39, 44, 32]) target is 49


In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # displays the occurence of all bigram possibilities a->b
        # each token mapped to a corresponding embedding vector
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # nn.Embedding is learnable parameter

    def forward(self, index, targets =None):
        #logits: a tensor where each row corresponds to the embedding vector of a token indexed by index
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            # b: batch size, t: sequence length (time), c: vocabulary size
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # view: same data as the self tensor but of a different shape
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # generate new tokens
    def generate(self, index, max_new_tokens):
        #index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            # bigram: focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim = -1)
            # sample from the multinomial distribution
            index_next = torch.multinomial(probs, num_samples = 1) # (B,1)
            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# attempting to generate without training
context = torch.zeros((1,1), dtype = torch.long, device = device) # single next line character
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


zDbTUc'0y(]m!]i
q?WUJ[Ek5i1,pkdE(TUv_dvE
Pte?XIf79;RKMfr_zM1PH?Bvr:oQmZQM1bD59hO1qK -D
73N"YlofOaF!OLNJcs0ksLKLhIW)x?LYPZ,Zph1bc9h5D,vK0f_*:tZao"7.?HNbY?roZ.! WXWTY!._;DGfHXq5ia0F9fmEOIK6'dW_Pq6gQM-1﻿pQc30](B,_mG5]7. WXq;J[XhPi1mG]7IuRZH wh(]8"DwM3Fb1[pBf7Bex5vCPint(vi17BEtQ([ITeAQiZLXqSTuV,Fc?X8Y﻿_C([ZQNh5[H?arfUw;3hABcEu37u?iKN?X81
V-8H_Pq5JOM899
SZHPv]J2]*Omi
7-y(RYPlmWVbIB8XqC*z3e3L7c9[XA)GGelR"65Oa-(?v,i'[fKlk5x!OOp5te(.QquA,nyZmg,u'Z;4]7yR ]OvjoJZL:Be?J﻿Jh1klJANcqU-k3v6mP!LjhOM1YpEC3L.﻿9
.


In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # switching model in evaluation mode (dropout, batch normalization is turned off)
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

# model training
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, training loss {losses['train']:.3f}, val loss {losses['val']:.3f} ")
    # sample a batch of data
    xb, yb = get_batch('train')

    # feeding both index and target into forward function
    logits, loss = model.forward(xb, yb) #.forward can be ommitted
    # make sure gradients do not add over time (previous gradients do not affect current gradient)
    optimizer.zero_grad(set_to_none=True) # set to none to occupy less space
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, training loss 4.863, val loss 4.882 
step: 250, training loss 4.827, val loss 4.841 
step: 500, training loss 4.762, val loss 4.765 
step: 750, training loss 4.671, val loss 4.708 
step: 1000, training loss 4.663, val loss 4.651 
step: 1250, training loss 4.598, val loss 4.599 
step: 1500, training loss 4.526, val loss 4.535 
step: 1750, training loss 4.491, val loss 4.470 
step: 2000, training loss 4.431, val loss 4.413 
step: 2250, training loss 4.386, val loss 4.385 
step: 2500, training loss 4.335, val loss 4.300 
step: 2750, training loss 4.260, val loss 4.277 
step: 3000, training loss 4.196, val loss 4.227 
step: 3250, training loss 4.159, val loss 4.169 
step: 3500, training loss 4.114, val loss 4.126 
step: 3750, training loss 4.067, val loss 4.092 
step: 4000, training loss 4.029, val loss 4.021 
step: 4250, training loss 3.970, val loss 3.971 
step: 4500, training loss 3.941, val loss 3.964 
step: 4750, training loss 3.901, val loss 3.930 
step: 5000, training loss 

In [15]:
# generate after training
context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


s-h5[
cH6Chk5_sk7z-9Xthi4B(Ahip2]p)owlQ('gT0"Ssj4bQkebrol
Cty?ro!*-s'serooi*L.
 PZHHW)W﻿Fql-9p﻿__Fi*owiZntbvK3Npo""Pt?zzOO]()[;[v,O﻿Zwhadr c,Wivhu aG]lvU81Y[BLbi1)nNNx1,
C)4[IKMnyIK1OaketCgthEanky()b9[xH&ouRST29[HVIK-Hwais?shdYL ht:9crLbooz8.)(Va9.SFK﻿ZLK3.!0G[;1ttece!V1Pq&Vp6Maipi,B7haz-(ARCIuEte(7.cVizEpth,vDo:E
Pg fKCEj
TER
Z-Opiz-kulaYXth:wid*thow1XTO1fs d"Oink8wYx b
"I WD lu noi15[8ER
FwhEthie6
 pr(x4LYDQL22g*[]gI;0﻿z1cVU![
*Q!H&4Z?incE467OO6-Y[s ?JB0memPFLK;f
[IEy-6lZLx,ca!OpAS6h w39skd df
